<a href="https://colab.research.google.com/github/SeyedMehdi110/Per-1/blob/main/Question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required libraries 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cvxpy as cp
from sklearn.metrics import accuracy_score 

In [74]:
#reading data
dataframe = pd.read_csv('train.csv',header = None)

dataframe[0].replace(0, -1,inplace = True)
y = dataframe[0]
X = dataframe.drop([0], axis=1)

#splitting data
data_train, data_val, label_train, label_val = train_test_split(X, y, test_size=0.42, random_state=1)

#importing test data
test_set = pd.read_csv('test.csv',header = None)
test_set[0].replace(0, -1,inplace = True)

label_test = test_set[0]
data_test = test_set.drop([0], axis=1)

data_train=np.array(data_train)
data_test=np.array(data_test)
data_val=np.array(data_val)

label_train = np.array(label_train)
label_test = np.array(label_test)
label_val = np.array(label_val)

In [75]:
#implementing Primal
def sign(x):
    if x < 0:
        return -1
    return 1
#training function
def svm_train_primal(data_train , label_train , regularisation_para_C ):
    data_train = np.c_[data_train, np.ones(data_train.shape[0])]
    n, col = data_train.shape  #assigng n and col to the shape of data
    w = cp.Variable(col) #defining the weight 
    e = cp.Variable(n)   # ξi 
    P = np.identity(col) #identity matrix
    q = np.ones(n)       # creating a vector of ones oadding ξi together
    b=cp.Variable()
    
    
    ##creating the primal problem using cvxpy
    prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(w, P) + (regularisation_para_C/n)*q.T @ e),
                     [-1*(label_train*(w.T @ data_train.T))-b*label_train<= e-1,
                      -e <= np.zeros(n)])
    prob.solve(verbose=True)


    return w.value,b.value,e.value
#prediction and calculating accuracy function
def svm_predict_primal(data_test , label_test , svm_model):
    data_test = np.c_[data_test, np.ones(data_test.shape[0])]
    pred = data_test @ svm_model
    prediction = []
    for i in pred:
        prediction.append(sign(i))
    prediction = np.array(prediction)
    acc = accuracy_score(label_test,prediction)
    return acc

In [76]:
svm_model = svm_train_primal ( data_train , label_train , 100 )

test_accuracy = svm_predict_primal ( data_test , label_test , svm_model[0] )


-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 5132, constraints m = 9860
          nnz(P) + nnz(A) = 1005921
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.6000e+03   1.60e+01   1.00e+01   1.00e-01   1.02e+00s
 125   2.2886e-09   5.05e-10   5.54e-14   2.12e-03   4.94e+00s
plsh   2.2980e-09   3.00e-16   2.28e-09

In [77]:
#printing out results
weights=svm_model[0]
b=svm_model[1]
train_acc = svm_predict_primal(data_train, label_train, weights)
val_acc = svm_predict_primal(data_val, label_val, weights)
test_acc = svm_predict_primal(data_test, label_test, weights)

print("Optimal b= ",b)
print("Sum of weights= ",np.sum(weights))
print("Train Accuracy= ",train_acc)
print("Validation Accuracy= ",val_acc)
print("Test Accuracy= ",test_acc)

Optimal b=  -3.070594966624755e-20
Sum of weights=  1.7743839584838565e-06
Train Accuracy=  0.8655172413793103
Validation Accuracy=  0.8588235294117647
Test Accuracy=  0.8666666666666667


In [8]:
weights[1]

1.67796532907901e-07

In [70]:
data_train = np.c_[data_train, np.ones(data_train.shape[0])]
e=svm_model[2]
n, col = data_train.shape
SVM=[]
for i in range(n):
  if np.abs((label_train[i]*(weights.T @ data_train[i].T))+b*label_train[i]-1+e[i])<= 0.0001:
    SVM.append(data_train[i])

In [71]:
SVM

[]